In [1]:
### all of the import libraries
import pygame
import math
import uuid
import time
import utility

from laser import Laser

from enemy import EnemyPlane

from pygame import sprite
from pygame import gfxdraw

from copy import deepcopy
from copy import copy
from random import randint

# constants
DEBUG = False
GAME_SHOULD_RUN = True

DISPLAY_WIDTH = 200
DISPLAY_HEIGHT = 100

BLACK = (0,0,0)
WHITE = (255,255,255)
RED = (255,0,0)

GRID_COLOR = (255,255,255)
X_GRID_SIZE = 192
Y_GRID_SIZE = 108

PLAYER_X_SIZE = 100
PLAYER_Y_SIZE = 100
PLAYER = pygame.image.load("./assets/sprites/player ship.png")
PLAYER = pygame.transform.scale(PLAYER,(PLAYER_X_SIZE, PLAYER_Y_SIZE))
PLAYER = pygame.transform.rotate(PLAYER,90)

RED_LASER = pygame.image.load("./assets/sprites/red laser.png")
RED_LASER = pygame.transform.scale(RED_LASER,(50,100))
RED_LASER = pygame.transform.rotate(RED_LASER,270)

BLUE_LASER = pygame.image.load("./assets/sprites/blue laser.png")
BLUE_LASER = pygame.transform.scale(BLUE_LASER,(50,100))
BLUE_LASER = pygame.transform.rotate(BLUE_LASER,270)

ENEMY_X_SIZE = 100
ENEMY_Y_SIZE = 100

ENEMY = pygame.image.load("./assets/sprites/red enemy.png")
ENEMY = pygame.transform.scale(ENEMY,(ENEMY_X_SIZE, ENEMY_Y_SIZE))
ENEMY = pygame.transform.rotate(ENEMY,90)

ENEMY2 = pygame.image.load("./assets/sprites/blue enemy.png")
ENEMY2 = pygame.transform.scale(ENEMY2,(ENEMY_X_SIZE, ENEMY_Y_SIZE))
ENEMY2 = pygame.transform.rotate(ENEMY2,90)
ENEMY_WAIT_TIME = 1000

# end of constants

# Initializes the game
pygame.init()
window = pygame.display.set_mode(size=(1920,1080), flags=pygame.FULLSCREEN) #Sets the size of the game window to the size of the screen

score = 0
start_time = time.time()
time_elapsed = 1
# sets the amount of hits the player can take
hit_points = 5
time_of_hit = time.time()

laser_fired = False

#player coordinates
player_x = X_GRID_SIZE
player_y = Y_GRID_SIZE + 5

enemy_x = 1920-ENEMY_X_SIZE/2
enemy_y = ENEMY_Y_SIZE/2

laser_x = player_x + 60
laser_y = player_y + 25
        
enemy_planes = EnemyPlane.create_planes(Y_GRID_SIZE, time_elapsed) 
lasers = []

# sets up a timer and des
timer = pygame.time.set_timer(pygame.USEREVENT, 1)

# sets a timer for the regeneration of the enemy planes.
enemy_regen_timer = pygame.time.set_timer(pygame.USEREVENT+1, 17)

laser_move_timer = pygame.time.set_timer(pygame.USEREVENT + 3, 21)


font = pygame.font.Font("./assets/fonts/font 1.ttf", 50)
scoreboard = font.render("Score: 0" , False, (255, 255, 255))
health = font.render("Health: 5", False, (255,255,255))

def blit_all():
    for enemy in enemy_planes: # updates the positions of the enemies on the screen by going through the list of designated positions
        if enemy.color == "red":
            window.blit(ENEMY,(enemy.x_pos,enemy.y_pos))
        elif enemy.color == "blue":
            window.blit(ENEMY2,(enemy.x_pos,enemy.y_pos))
        
    
    window.blit(PLAYER,(player_x,player_y)) # updates the position of the player plane using the x and y
    
    for laser in lasers:
        if laser.x_pos >= 1921:
            Laser.remove_laser(laser.id, lasers)
        if laser.color == "red":
            window.blit(RED_LASER, (laser.x_pos, laser.y_pos))
        if laser.color == "blue":
            window.blit(BLUE_LASER, (laser.x_pos, laser.y_pos))
    window.blit(scoreboard, (200,925))
    window.blit(health,(200,100))

def player_key_presses(event):
    global player_y
    global player_x
    
    if event.type == pygame.KEYDOWN: # plane move up
        
        if event.key == pygame.K_w:
            player_y -= Y_GRID_SIZE
            if player_y <= 102:
                player_y += Y_GRID_SIZE

        if event.key == pygame.K_s: # plane move down
            player_y += Y_GRID_SIZE
            if player_y >= 976:
                player_y -= Y_GRID_SIZE

        if event.key == pygame.K_d: # plane move left
            player_x += X_GRID_SIZE
            if player_x > 1728:
                player_x -= X_GRID_SIZE

        if event.key == pygame.K_a: #plane move right
            player_x -= X_GRID_SIZE
            if player_x < 192:
                player_x += X_GRID_SIZE
        
        if event.key == pygame.K_q: # adds laser if space pressed
            lasers.append(Laser((player_x + 60, player_y + 25), "red"))
            
        if event.key == pygame.K_e:
            lasers.append(Laser((player_x + 60, player_y + 25), "blue"))

# makes all of the game code while game_should_run is true
while GAME_SHOULD_RUN:
    background_color = (0,0,0)
    window.fill(background_color)
    time_elapsed = time.time()-start_time
    
    if DEBUG:
        utility.draw_grid(window, X_GRID_SIZE, Y_GRID_SIZE)
    # events
    
    
    for event in pygame.event.get():
        
        player_key_presses(event)
        
        if event.type == pygame.USEREVENT+1: # when the enemy regen timer goes off this regens enemies
            for plane in enemy_planes:
                if plane.x_pos < -50:
                    plane._x_pos = 1870
                    plane._y_pos = randint(1,8) * 108+4
        
        if event.type == pygame.USEREVENT: # when the enemy move timer goes off the enemies move.
            
            for plane in enemy_planes:
                plane._x_pos -= plane.speed
            enemy_copy = copy(enemy_planes)    
            for enemy in enemy_copy: # creates a copy of enemy_planes that we run through without changing it
                
                enemy_rect = utility.get_rect(ENEMY, enemy) # gets a rectangle around the enemy
                player_rect = PLAYER.get_rect(center = (player_x, player_y)) # gets a rectangle around the player's plane
                laser_copy = copy(lasers)
                
                for laser in laser_copy: #creates a copy of lasers that we can run through without changing it
                    laser_rect = utility.get_rect(RED_LASER, laser) # gets a rectangle around the laser
                    
                    if laser_rect.colliderect(enemy_rect) == True: # If the rectangle of the laser is touching the rectangle of the enemy, this return True
                        
                        if laser.color == "red" and enemy.color == "red" or laser.color == "blue" and enemy.color == "blue":
                            lasers.remove(laser)
                            
                            if enemy in enemy_planes:
                                enemy_planes.remove(enemy)
                                enemy_planes.append(EnemyPlane(Y_GRID_SIZE, time_elapsed))
                                score += 100
                                scoreboard = font.render("Score: " + str(score) , False, (255, 255, 255))
                        
                        elif laser.color == "red" and enemy.color == "blue" or laser.color == "blue" and enemy.color == "red":
                            lasers.remove(laser)
                            hit_points -= 1
                            health = font.render("Health: " + str(hit_points), False, (255,255,255))
                            if hit_points == 0:
                                GAME_SHOULD_RUN = False
                                
                if player_rect.colliderect(enemy_rect) == True:
                    if time.time() - time_of_hit >= 3:
                        hit_points -= 1
                        time_of_hit = time.time()
                        health = font.render("Health: " + str(hit_points), False, (255,255,255))
                        if hit_points == 0:
                            GAME_SHOULD_RUN = False
        if event.type == pygame.USEREVENT + 3:
            for laser in lasers:
                laser.movement()
                
        if event.type == pygame.QUIT: # stops the game if it recieves the event QUIT
            GAME_SHOULD_RUN = False
            break
            
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                GAME_SHOULD_RUN = False
                break
    
    blit_all()
    pygame.display.update() # finalizes the updates

# ends the game and exits the tab
pygame.quit()
exit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
